In [1]:
# Joey Roberts - Amath 481 - HW5

import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from scipy.sparse import spdiags
from scipy.integrate import solve_ivp
from scipy.linalg import lu, solve_triangular
from scipy.sparse.linalg import bicgstab
from scipy.sparse.linalg import gmres
from matplotlib.animation import FuncAnimation, PillowWriter

# Part A

# Define parameters
tspan = np.arange(0, 4.5, 0.5)  # Time points
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64  # Grid size
N = nx * ny

# Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)  # 65 points for periodic BC
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
# Parameters for the elliptical Gaussian
w2 = 1 * np.exp((-X**2) - (Y**2)/20) + 1j * np.zeros((nx, ny))
w = w2.reshape(N)

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6  # Avoid division by zero
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

m = nx  # N value in x and y directions
n = m * m  # Total matrix size
dx = 20 / m  # Spacing of grid

# Define vectors for diagonals
d1 = np.ones(n)
d2 = np.ones(n)
d4 = np.zeros(n)

for j in range(1, m+1):
    d2[m*j-1] = 0
    d4[m*j-1] = 1

d3 = np.roll(d2, 1)
d5 = np.roll(d4, 1)

# Matrix A
diagonals = [d1, d1, d5, d2, -4 * d1, d3, d4, d1, d1]
offsets = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
A = spdiags(diagonals, offsets, n, n) / dx**2
A = A.toarray()
A[0,0] = 2

# Matrix B
diagonals = [d1, -d1, d1, -d1]
offsets = [-(n-m), -m, m, (n-m)]
B = spdiags(diagonals, offsets, n, n) / (2*dx)
B = B.toarray()

# Matrix C
diagonals = [d5, -d2, d3, -d4]
offsets = [-m+1, -1, 1, m-1]
C = spdiags(diagonals, offsets, n, n) / (2*dx)
C = C.toarray()

# Define the ODE system
def spc_rhs(t, w, nx, ny, N, KX, KY, K, nu):
    wc2 = w.reshape((nx, ny))
    wtc2 = fft2(wc2)
    psit = -wtc2 / K
    psi = np.real(ifft2(psit)).reshape(N)  # Real-space psi, flattened
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs 

# Solve the ODE
solution = solve_ivp(spc_rhs, [0, 4], w, t_eval=tspan, args=(nx, ny, N, KX, KY, K, nu), method="RK45")

A1 = solution.y
print(A1)

[[2.50656748e-46+0.j 3.57995003e-45+0.j 1.88183146e-44+0.j ...
  1.89041895e-42+0.j 4.77633194e-42+0.j 1.13650581e-41+0.j]
 [1.17762859e-43+0.j 6.53073985e-43+0.j 2.62225766e-42+0.j ...
  1.58766537e-40+0.j 3.56176571e-40+0.j 7.46726173e-40+0.j]
 [4.55107657e-41+0.j 1.92911214e-40+0.j 6.43115427e-40+0.j ...
  2.49164423e-38+0.j 5.07836881e-38+0.j 9.69130857e-38+0.j]
 ...
 [1.96785570e-38+0.j 1.23433506e-37+0.j 5.47709461e-37+0.j ...
  4.36374689e-35+0.j 1.02587928e-34+0.j 2.23425898e-34+0.j]
 [6.19028421e-41+0.j 5.33867748e-40+0.j 2.93948956e-39+0.j ...
  3.69117060e-37+0.j 9.47855834e-37+0.j 2.24193485e-36+0.j]
 [1.60178709e-43+0.j 1.99617096e-42+0.j 1.41293854e-41+0.j ...
  2.81079660e-39+0.j 7.98089278e-39+0.j 2.08143727e-38+0.j]]


In [2]:
# Part B

w2 = 1 * np.exp((-X**2) - (Y**2)/20)
w = w2.reshape(N)

# Define the ODE system
def gas_rhs(t, w, nu):
    psi = np.linalg.solve(A, w)  # Real-space psi, flattened
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs 

# Solve the ODE
solution = solve_ivp(gas_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")

A2 = solution.y
print(A2)

[[ 2.50656748e-46 -1.78784704e-36  1.02706391e-36 ... -5.76806489e-26
  -6.68287054e-25  3.02050167e-24]
 [ 1.17762859e-43 -2.26093387e-29  1.85614646e-29 ... -1.48306461e-19
  -7.82424220e-19  1.67449651e-18]
 [ 4.55107657e-41 -4.24661881e-29  3.48720993e-29 ... -2.20165937e-20
  -2.91637720e-19  1.23961335e-18]
 ...
 [ 1.96785570e-38  7.20814729e-28 -6.58759786e-28 ...  4.01674620e-20
   1.34371413e-19 -8.94163775e-19]
 [ 6.19028421e-41  1.69117621e-28 -1.49205998e-28 ...  3.00460834e-20
   1.76009486e-19 -9.80736832e-19]
 [ 1.60178709e-43  5.05834665e-29 -4.38046443e-29 ...  2.47633369e-20
   1.15181522e-20  3.12761254e-20]]


In [3]:
# Part C

# Define the ODE system
def lu_rhs(t, w, nu):
    P, L, U = lu(A)
    Pw = np.dot(P, w)
    y = solve_triangular(L, Pw, lower=True)
    psi = solve_triangular(U, y)
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs 

# Solve the ODE
solution = solve_ivp(lu_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")

A3 = solution.y
print(A3)

[[ 2.50656748e-46 -1.78784704e-36  1.02706391e-36 ... -5.76806489e-26
  -6.68287054e-25  3.02050167e-24]
 [ 1.17762859e-43 -2.26093387e-29  1.85614646e-29 ... -1.48306461e-19
  -7.82424220e-19  1.67449651e-18]
 [ 4.55107657e-41 -4.24661881e-29  3.48720993e-29 ... -2.20165937e-20
  -2.91637720e-19  1.23961335e-18]
 ...
 [ 1.96785570e-38  7.20814729e-28 -6.58759786e-28 ...  4.01674620e-20
   1.34371413e-19 -8.94163775e-19]
 [ 6.19028421e-41  1.69117621e-28 -1.49205998e-28 ...  3.00460834e-20
   1.76009486e-19 -9.80736832e-19]
 [ 1.60178709e-43  5.05834665e-29 -4.38046443e-29 ...  2.47633369e-20
   1.15181522e-20  3.12761254e-20]]


In [4]:
'''

# BiCGSTAB

# Define the ODE system using BiCGSTAB
def BIG_rhs(t, w, nu):
    # Solve A * psi = w using BiCGSTAB
    psi, exit_code = bicgstab(A, w, tol=1e-6)
    
    if exit_code != 0:
        raise ValueError(f"BiCGSTAB failed to converge. Exit code: {exit_code}")
    
    # Compute the right-hand side
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs

# Solve the ODE
solution = solve_ivp(BIG_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")

BIG = solution.y
print(BIG)

'''

'\n\n# BiCGSTAB\n\n# Define the ODE system using BiCGSTAB\ndef BIG_rhs(t, w, nu):\n    # Solve A * psi = w using BiCGSTAB\n    psi, exit_code = bicgstab(A, w, tol=1e-6)\n    \n    if exit_code != 0:\n        raise ValueError(f"BiCGSTAB failed to converge. Exit code: {exit_code}")\n    \n    # Compute the right-hand side\n    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))\n    return rhs\n\n# Solve the ODE\nsolution = solve_ivp(BIG_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")\n\nBIG = solution.y\nprint(BIG)\n\n'

In [5]:
'''

# GMRES

# Define the ODE system using GMRES
def GM_rhs(t, w, nu):
    # Solve A * psi = w using GMRES
    psi, exit_code = gmres(A, w, tol=1e-6)
    
    if exit_code != 0:
        raise ValueError(f"GMRES failed to converge. Exit code: {exit_code}")
    
    # Compute the right-hand side
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs

# Solve the ODE
solution = solve_ivp(GM_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")

GMsol = solution.y
print(GMsol)

'''

'\n\n# GMRES\n\n# Define the ODE system using GMRES\ndef GM_rhs(t, w, nu):\n    # Solve A * psi = w using GMRES\n    psi, exit_code = gmres(A, w, tol=1e-6)\n    \n    if exit_code != 0:\n        raise ValueError(f"GMRES failed to converge. Exit code: {exit_code}")\n    \n    # Compute the right-hand side\n    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))\n    return rhs\n\n# Solve the ODE\nsolution = solve_ivp(GM_rhs, [0, 4], w, t_eval=tspan, args=(nu,), method="RK45")\n\nGMsol = solution.y\nprint(GMsol)\n\n'

In [7]:


# Define parameters
tspan = np.arange(0, 30.5, 0.5)  # Time points

# Parameters for the elliptical Gaussian
#w2 = -3 * np.exp((-(X-1)**2/2) - (Y**2/2)) + 3 * np.exp((-(X+1)**2/2) - (Y**2/2)) + 1j * np.zeros((nx, ny))
w2 = 3 * np.exp((-(X-2)**2/2) - (Y**2/2)) + 3 * np.exp((-(X+2)**2/2) - (Y**2)) + 1j * np.zeros((nx, ny))
#w2 = 2 * np.exp((-X**2) - (Y**2)/20) - 1 * np.exp((-X**2/10) - (Y**2)) + 1j * np.zeros((nx, ny))
#w2 = 0.5 * np.exp((-(X)**2/2) - (Y**2/40))-2 * np.exp((-(X+2)**2/3) - ((Y+1)**2/3))+1 * np.exp((-(X-5)**2) - ((Y+1)**2))-1 * np.exp((-(X-8)**2/3) - ((Y-3)**2))+3 * np.exp((-(X+3)**2/2) - ((Y-2)**2))-1 * np.exp((-(X+6)**2) - ((Y-3)**2/10))+2 * np.exp((-(X-1)**2/8) - ((Y-4)**2/3))-3 * np.exp((-(X+1)**2/4) - ((Y+6)**2/1))+3 * np.exp((-(X-2)**2/2) - ((Y+8)**2/7))-1 * np.exp((-(X-3)**2) - ((Y-2)**2))-2 * np.exp((-(X-4)**2/2) - ((Y+9)**2/3))+1 * np.exp((-(X-7)**2) - ((Y-2)**2/6)) + 1j * np.zeros((nx, ny))
w = w2.reshape(N)

# Define parameters
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 128, 128  # Grid size
N = nx * ny

# Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)  # 65 points for periodic BC
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Parameters for the elliptical Gaussian
#w2 = -3 * np.exp((-(X-1)**2/2) - (Y**2/2)) + 3 * np.exp((-(X+1)**2/2) - (Y**2/2)) + 1j * np.zeros((nx, ny))
w2 = 3 * np.exp((-(X-2)**2/2) - (Y**2/2)) + 3 * np.exp((-(X+2)**2/2) - (Y**2)) + 1j * np.zeros((nx, ny))
#w2 = 2 * np.exp((-X**2) - (Y**2)/20) - 1 * np.exp((-X**2/10) - (Y**2)) + 1j * np.zeros((nx, ny))
#w2 = 0.5 * np.exp((-(X)**2/2) - (Y**2/40))-2 * np.exp((-(X+2)**2/3) - ((Y+1)**2/3))+1 * np.exp((-(X-5)**2) - ((Y+1)**2))-1 * np.exp((-(X-8)**2/3) - ((Y-3)**2))+3 * np.exp((-(X+3)**2/2) - ((Y-2)**2))-1 * np.exp((-(X+6)**2) - ((Y-3)**2/10))+2 * np.exp((-(X-1)**2/8) - ((Y-4)**2/3))-3 * np.exp((-(X+1)**2/4) - ((Y+6)**2/1))+3 * np.exp((-(X-2)**2/2) - ((Y+8)**2/7))-1 * np.exp((-(X-3)**2) - ((Y-2)**2))-2 * np.exp((-(X-4)**2/2) - ((Y+9)**2/3))+1 * np.exp((-(X-7)**2) - ((Y-2)**2/6)) + 1j * np.zeros((nx, ny))
w = w2.reshape(N)

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6  # Avoid division by zero
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

m = nx  # N value in x and y directions
n = m * m  # Total matrix size
dx = 20 / m  # Spacing of grid

# Define vectors for diagonals
d1 = np.ones(n)
d2 = np.ones(n)
d4 = np.zeros(n)

for j in range(1, m+1):
    d2[m*j-1] = 0
    d4[m*j-1] = 1

d3 = np.roll(d2, 1)
d5 = np.roll(d4, 1)

# Matrix A
diagonals = [d1, d1, d5, d2, -4 * d1, d3, d4, d1, d1]
offsets = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
A = spdiags(diagonals, offsets, n, n) / dx**2
A = A.toarray()
A[0,0] = 2

# Matrix B
diagonals = [d1, -d1, d1, -d1]
offsets = [-(n-m), -m, m, (n-m)]
B = spdiags(diagonals, offsets, n, n) / (2*dx)
B = B.toarray()

# Matrix C
diagonals = [d5, -d2, d3, -d4]
offsets = [-m+1, -1, 1, m-1]
C = spdiags(diagonals, offsets, n, n) / (2*dx)
C = C.toarray()

# Define the ODE system
def spc_rhs(t, w, nx, ny, N, KX, KY, K, nu):
    wc2 = w.reshape((nx, ny))
    wtc2 = fft2(wc2)
    psit = -wtc2 / K
    psi = np.real(ifft2(psit)).reshape(N)  # Real-space psi, flattened
    rhs = (nu * A.dot(w) - B.dot(psi) * C.dot(w) + C.dot(psi) * B.dot(w))
    return rhs 

# Solve the ODE
solution = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w, t_eval=tspan, args=(nx, ny, N, KX, KY, K, nu), method="RK45")

sol = solution.y

#GIF creation

# Prepare frames for the GIF
frames = []
fig, ax = plt.subplots(figsize=(6, 6))
pcm = ax.pcolor(x, y, np.real(w2), shading="auto", cmap="magma")
plt.colorbar(pcm, ax=ax)
ax.set_title("Vorticity Evolution (A1)")

def update(frame_idx):
    w = np.real((sol[:, frame_idx].reshape((nx, ny))))
    pcm.set_array(w.ravel())  # Update plot data
    ax.set_title(f"t = {tspan[frame_idx]:.1f}")
    return pcm,

ani = FuncAnimation(fig, update, frames=len(tspan), blit=False)

# Change the path to a location on your machine
gif_path = "vorticity_evolution_same2.gif" 
ani.save(gif_path, writer=PillowWriter(fps=15))
plt.close(fig)

print(f"GIF saved at {gif_path}")


GIF saved at vorticity_evolution_same2.gif
